# bikeの特徴量の解釈
https://christophm.github.io/interpretable-ml-book/limo.html

## データの意味

- cnt:casualとregisteredのユーザーの数。回帰のターゲット
- season:spring, summer, fall or winter.
- holiday
- year: 2011 or 2012.
- dateday:2011/01/01からの日数。この特料量は時間経過によるトレンドを導くのに使用する。
- working day or weekend.
- 天気:
    - 1 clear, few clouds, partly cloudy, cloudy
    - 2 mist + clouds, mist + broken clouds, mist + few clouds, mist
    - 3 light snow, light rain + thunderstorm + scattered clouds, light rain + scattered clouds
    - 4 heavy rain + ice pallets + thunderstorm + mist, snow + mist
- temp：Temperature in degrees Celsius.41で割り、正規化されている。
- atemp：体感温度。50で割り、正規化されている。
- hum:湿度(0 to 100). 100で割り正規化されている。
- Wind speed: km/hour.　67で割り、正規化されている。

In [1]:
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import pandas as pd
import seaborn as sns;sns.set()
%matplotlib inline
os.getcwd()

'C:\\home\\myproj\\interpretable-machine-learning\\LinearRegression\\FeatureImportance'

In [2]:
data_dir = '../../data/bike'
day_file = 'day.csv'
hour_file = 'hour.csv'

In [3]:
df_day = pd.read_csv(os.path.join(data_dir, day_file))
# headにすることで先頭だけ確認できる。
df_day.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [4]:
# ここからデータを整形していく
# 目的変数をYに
Y = df_day['cnt']

#　dteday をなくしt、instantをdayに（本当は日付を経過日数に変換しないと駄目）
del df_day['dteday']
del df_day['yr']

df_day = df_day.rename(columns={'instant': 'days'})

# onehot(季節)
encoder = OneHotEncoder()
enced = encoder.fit_transform(df_day.season.values.reshape(1, -1).transpose())
df_season = pd.DataFrame(index=df_day.season.index, columns=['season-SPRING', 'season-SUMMER', 'season-FALL', "season-WINTER"], data=enced.toarray())
df_day = pd.concat([df_day, df_season], axis=1)
del df_day['season']

# onehot(天気)
encoder = OneHotEncoder()
enced = encoder.fit_transform(df_day.weathersit.values.reshape(1, -1).transpose())
df_weather = pd.DataFrame(index=df_day.weathersit.index, columns=['weather-CLEAR', 'weather-MYST', 'season-SNOW'], data=enced.toarray())
df_day = pd.concat([df_day, df_weather], axis=1)
del df_day['weathersit']

# 正規化を戻す
df_day['temp'] = df_day['temp'] *  (39 - (-8)) + (-8)
df_day['hum'] = df_day['hum'] * 100
df_day['windspeed'] = df_day['windspeed'] * 67

# 未使用の特徴量を削除
del df_day['casual']
del df_day['registered']
del df_day['cnt']
del df_day['mnth']
del df_day['weekday']
del df_day['atemp']

# 最終的な特徴量
df_day.head()

,days,holiday,workingday,temp,hum,windspeed,season-SPRING,season-SUMMER,season-FALL,season-WINTER,weather-CLEAR,weather-MYST,season-SNOW
0,1,0,0,8.175849,80.5833,10.749882,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0,0,9.083466,69.6087,16.652113,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,0,1,1.229108,43.7273,16.636703,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0,1,1.400000,59.0435,10.739832,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0,1,2.666979,43.6957,12.522300,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
# モデルの学習
# sklearnだとt値の計算方法が分からなかった
model = sm.OLS(Y, df_day)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     251.2
Date:                Wed, 03 Apr 2019   Prob (F-statistic):          1.05e-237
Time:                        10:05:14   Log-Likelihood:                -5993.0
No. Observations:                 731   AIC:                         1.201e+04
Df Residuals:                     719   BIC:                         1.207e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
days              4.9264      0.173     28.507      0.000       4.587       5.266
holiday        -686.1154    203.301     -3.375      0.001   -1085.251    -286.980
workingday      124.9209     73.267      1.705      0.089     -18.921     268.763
temp            110.7096      7.043     15.718      0.000      96.882     124.537
hum             -17.3772      3.169     -5.483      0.000     -23.600     -11.155
windspeed       -42.5135      6.892     -6.169      0.000     -56.044     -28.983
season-SPRING   491.3532    134.637      3.649      0.000     227.025     755.682
season-SUMMER  1390.6714    143.722      9.676      0.000    1108.507    1672.836
season-FALL     629.5686    166.154      3.789      0.000     303.364     955.773
season-WINTER   916.9561    146.517      6.258      0.000     629.304    1204.608
weather-CLEAR  1903.1626    133.726     14.232      0.000    1640.623    2165.702
weather-MYST   1523.7640    173.469      8.784      0.000    1183.198    1864.330
season-SNOW       1.6227    267.240      0.006      0.995    -523.042     526.287
==============================================================================
Omnibus:                       91.525   Durbin-Watson:                   0.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              194.706
Skew:                          -0.719   Prob(JB):                     5.25e-43
Kurtosis:                       5.079   Cond. No.                     2.33e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.44e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""